In [77]:
import torch
import torch.nn as nn
import torch.functional as F
import math

In [78]:
# 多头注意力
# 创建模型结构
# 问题1：前面这一这段代码的含义是什么，python相关
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_head, is_mask = False):
        # 注意这里super要传入自己和self，不能只传入自己，你要多学一些python的基础知识了
        super(MultiHeadAttention, self).__init__()
        
        # 初始化超参数
        self.n_head = n_head
        self.d_model = d_model
        # 创建qkv的三个线性映射层，用一个矩阵和qkv相乘，让qkv变得可学习
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        # 问题2：这个combine的作用是什么，矩阵经过怎样的运算
        self.combine = nn.Linear(d_model,d_model)
        self.softmax = nn.Softmax(dim=-2)

    def forward(self, q, k, v):
        batch, time, dimension = q.shape
        # 得到每个头的dimension
        n_d = d_model // n_head
        # 让qkv进入线性层
        q, k, v = self.w_q(q), self.w_k(k), self.w_v(v)

        # 拆分qkv
        # 问题3:为什么需要permute这个东西： 
        # 问题4:这个转置函数怎么用的，结果是什么
        # 问题5:这两个四维矩阵是怎么乘起来的
        # 为了更好的并行计算，变换之后，qkv矩阵形状变成(batch, n_head, time, n_d)，之后k要再次变换成(batch, n_head, n_d, time)
        # 最后q @ k结果的shape是(batch, n_head, time, time)两个四维矩阵相乘，前面两个维度都不参与运算。
        q = q.view(batch, time, n_head, n_d).permute(0, 2, 1, 3)
        k = k.view(batch, time, n_head, n_d).permute(0, 2, 1, 3)
        v = v.view(batch, time, n_head, n_d).permute(0, 2, 1, 3)
        # 都是交换维度，只是交换维度的数量区别
        # 计算注意力结果 q*k转置/n_d
        # score的shape为(batch, n_head, time, time)
        score = q @ k.transpose(2,3) / math.sqrt(n_d)
        # 生成mask，一个左下角为1的下三角矩阵
        if is_mask:
            mask = torch.tril(torch.ones(time, time, dtype=bool))
        # 让mask掩盖score中不应该被注意的数值,全部变成负无穷
        # 问题6：这个masked_fill函数是哪里的，矩阵自带的吗，怎么用， pytorch张量自带的方法，用于用特定值填充掩码位置。
        # 为什么需要combine
        # 在多头注意力机制中，将多个头的输出重新组合成一个张量后，还需要通过一个线性变换来生成最终的输出。这是因为每个头的输出仅仅代表该头的注意力结果，最后通过线性层来融合各个头的信息，生成最终的表示。
        # 1. 线性变换：尽管 permute 和 view 将多个头的输出重新组合成了一个张量，但是每个头的输出还需要进一步线性变换，以便在融合信息的同时对其进行适当的缩放和变换。
        # 2. 参数化：combine 层通过线性层实现参数化，使得整个模型能够学习如何将多个头的注意力结果组合成最终的输出表示。这是模型学习的关键部分。
        # 3. 增加模型表达能力：线性层通过增加可学习参数，提升了模型的表达能力，使其能够更好地捕捉复杂的模式和关系。
            score = score.masked_fill(mask == 0, float('-inf'))
        # 经过softmax然后@v, 点乘之后最后两个维度相乘，又变成(batch, n_head, time, n_d)了
        score = self.softmax(score) @ v

        # 把score的形状变换回来
        score = score.permute(0, 2, 1, 3).contiguous().view(batch, time, dimension)

        # combine score
        # 问题7：这个combine的作用是什么
        output = self.combine(score)
        return output

In [79]:
class TokenEmbedding(nn.Embedding):
    def __init__(self, vocab_size, d_model):
        super().__init__(vocab_size, d_model, padding_idx=1)

In [80]:
class PositionEncoder(nn.Module):
    # 输入为词向量长度，序列最大长度，设备
    def __init__(self, d_model, maxlen):
        # 初始化编码
        self.encoding = torch.zeros(maxlen, d_model)
        self.encoding.requires_grad = False
        # 初始化位置
        pos = torch.arange(0, maxlen)
        # 这里为什么要变成二维的
        pos = pos.float().unsqueeze(1)
        _2i = torch.arange(0, d_model, 2)
        # 广播机制，一个(5,1)的数组/（2）的数组，结果为（5，2）的数组
        # 生成位置编码
        self.encoding[:, 0::2] = torch.sin(pos / (10000 ** (_2i / d_model)))
        self.encoding[:, 1::2] = torch.cos(pos / (10000 ** (_2i / d_model)))
    # 前向传播
    def forward(self, x):
        len = x.shape[1]
        return self.encoding[:len, :]

In [81]:
class LayerNorm(nn.Module):
    def __init__(self, d_model, eps=1e-10):
        super().__init__()
        self.gamma = nn.Pameter(torch.ones(d_model))
        self.beta = nn.Pameter(torch.zeros(d_model))
        self.eps = eps
    
    def forward(self, x):
        # x最后一个维度的均值和方差，也就是d_model
        mean = x.mean(-1, keepdim=True)
        var = x.var(-1, unbiased = False, keepdim=True)
        out = x - mean / torch.sqrt(var + self.eps)
        out = self.gamma * out + self.beta
        return out
        

In [82]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden_size, drop_out = 0.1):
        super().__init__()
        fc1 = nn.Linear(d_model, hidden_size)
        fc1 = nn.Linear(hidden_size, d_model)
        drop_out = nn.Dropout(drop_out)
    
    def forward(self, x):
        # 第一层 -> relu -> drop_out -> 第二层
        x = fc1(x)
        x = drop_out(F.relu(x))
        return fc2(x)

# Total Embedding

In [83]:
class TotalEmbedding(nn.Module):
    def __init__(self, vocab_size, d_model, maxlen, drop_out = 0.1):
        super().__init__()
        # 词嵌入层
        self.token_embedding = TokenEmbedding(vocab_size, d_model)
        self.position_encoder = PositionEncoder(d_model, maxlen)
        self.drop_out = nn.Dropout(drop_out)

    def forward(self, x):
        # 词嵌入层 -> 位置编码层 -> drop_out
        tok_embedding = self.token_embedding(x)
        pos_encode = self.position_encoder(x)
        return drop_out(tok_embedding + pos_encode)

# Encoder

In [84]:
class Encoder(nn.Module):
    def __init__(self, d_model, maxlen, hidden_size, drop_out = 0.1):
        self.multi_head_attention = MultiHeadAttention(d_model, hidden_size, is_mask=False)
        self.layer_norm1 = LayerNorm(d_model)
        self.drop1 = nn.Dropout(drop_out)
        
        self.layer_norm2 = LayerNorm(d_model)
        self.drop2 = nn.Dropout(drop_out)
        self.position_wise_feed_forward = PositionWiseFeedForward(d_model, hidden_size, drop_out)
    
    def forward(self, x):
        # 多头注意力 -> 残差连接 -> 层归 -> 位置前馈 -> 残差连接 -> 层归
        _x = x
        x = self.multi_head_attention(x)
        x = self.layer_norm1(self.drop1(x))
        x = _x + x

        _x = x
        x = self.position_wise_feed_forward(x)
        x = self.layer_norm2(self.drop2(x))
        return x + _x 

        

In [48]:
x = torch.arange(0, 4)
x = x.float().unsqueeze(1)
print(x.shape)
y = torch.arange(1, 3)
print(y.shape)
print(x / y)

torch.Size([4, 1])
torch.Size([2])
tensor([[0.0000, 0.0000],
        [1.0000, 0.5000],
        [2.0000, 1.0000],
        [3.0000, 1.5000]])


In [57]:
x = torch.randn(2, 2, 3)
mean = x.mean(-1, keepdim=True)
print(x.shape)
mean.shape

torch.Size([2, 2, 3])


torch.Size([2, 2, 1])